In [10]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
from pymongo import Connection
connection = Connection()#這是在自己的本機建的，所以用connection

In [11]:
db = connection['Candidate']
db.collection_names(include_system_collections=False) #listing all of the collections in our database
# collection = db['comments']

[u'comments', u'posts', u'reactions', u'detector', u'detector_yes']

In [12]:
facebook_id = '46251501064' #Tsai
facebook_page_name = 'Tsai'
# facebook_id = '136845026417486' #Ke
# facebook_page_name = 'Ke'
# facebook_id = '188311137478' #ChinaTimes 
# facebook_page_name = 'ChinaTimes'
# facebook_id = '241284961029'
# facebook_page_name = 'udn' 
# facebook_id = '394896373929368' 
# facebook_page_name = 'ltn' 
# facebook_id = '232633627068' 
# facebook_page_name = 'AppleDaily'

##一篇一篇標記

In [28]:
import string
import json

tsai_sup=['蔡總統加油','小英總統','英英總統','蔡英文總統加油','女總統加油','阿英總統加油','蔡總統我支持你','小英總統我支持你','英英總統我支持你','蔡英文總統我支持你','女總統我支持你','阿英總統我支持你']
tsai_not_sup=['台奸蔡英文','英英美代子','白賊英','蔡英文下台','蔡英文爛死了','菜英文','空心菜','祭央乂','暴力小英','武則英','尤達大師','女版馬英九','雙面蔡','蔡渣','18趴小英','亡國小英']
# not_sure_1:['小英','阿英','小英總統','阿英總統','女總統','蔡總統','蔡英文','蔡英文總統','英英','英英總統']
# not_sure_2:['加油','讚','我愛你','支持你','我們支持你','全家支持你','我支持你']

cannot = []
user_dict={}

for c in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017}}):
    if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
#         把user_id存起來
        if not user_dict.has_key(c['user_id']):
            user_dict[c['user_id']]='1'
        else:
            user_dict[c['user_id']]+=',1'
#                 print c['comment_id']#c['comment]為unicode 
        db.detector_one_by_one.insert({'comment':c['comment'],
                                'user_id':c['user_id'],
                                'user_name':c['user_name'],
                                'comment_id':c['comment_id'],
                                'post_id':c['post_id'],
                                'page_name':facebook_page_name,
                                'page_id':facebook_id,
                                'comment_count':c['comment_count'],
                                'like_count':c['like_count'],
                                'stand':1})
    elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in tsai_not_sup):
            if not user_dict.has_key(c['user_id']):
                user_dict[c['user_id']]='-1'
            else:
                user_dict[c['user_id']]+=',-1'

            db.detector_one_by_one.insert({'comment':c['comment'],
                                    'user_id':c['user_id'],
                                    'user_name':c['user_name'],
                                    'comment_id':c['comment_id'],
                                    'post_id':c['post_id'],
                                    'page_name':facebook_page_name,
                                    'page_id':facebook_id,
                                    'comment_count':c['comment_count'],
                                    'like_count':c['like_count'],
                                    'stand':-1,
                                    'auto':0})
    else:
        if not user_dict.has_key(c['user_id']):
            user_dict[c['user_id']]='0'
        else:
            user_dict[c['user_id']]+=',0'

        db.detector_one_by_one.insert({'comment':c['comment'],
                        'user_id':c['user_id'],
                        'user_name':c['user_name'],
                        'comment_id':c['comment_id'],
                        'post_id':c['post_id'],
                        'page_name':facebook_page_name,
                        'page_id':facebook_id,
                        'comment_count':c['comment_count'],
                        'like_count':c['like_count'],
                        'stand':0,
                        'auto':0})

    
db.detector_one_by_one.count()
db.update
# comments.update()
print db.detector_one_by_one.count()
# db.posts.find({})

#把user_dict存成json檔
with open('user_one_by_one.json', 'w') as f:
    json.dump(user_dict, f)

821725


NameError: name 'json' is not defined

###Finding average of a list: http://stackoverflow.com/questions/9039961/finding-the-average-of-a-list

In [30]:
#找出共有幾位留言者
print len(user_dict)

240567


In [45]:
# 計算這個留言者支持跟不支持的數值，小數位取到第一位
count={}
for u,i in user_dict.items():
#     i = map(int, i.translate(None, string.punctuation))  ###這裡會把負號弄掉！！！！！！
    i = [int(x.strip()) for x in i.split(',')]  #所以改成這樣
    h= float(reduce(lambda q,p: p+q, i)/float(len(i)))
    k = round(h,2)
#     if len(str(h).split('.')[1])>0:
    if not count.has_key(k):
        count[k]=1
    count[k]+=1
#     print u,h #不要印！！！！！！！！

In [46]:
# 印出來看看
for x, c in count.items():
    print x, c

0.0 230903
0.5 1318
0.25 587
1.0 2846
0.19 18
0.12 20
0.73 2
0.75 27
0.48 6
0.13 225
0.67 139
-0.83 3
0.24 16
0.69 3
0.44 13
-0.09 19
0.42 7
-0.06 26
0.92 2
0.85 2
0.05 128
0.1 127
0.8 4
0.65 3
0.39 3
0.71 6
0.38 26
0.11 169
-0.94 2
0.47 7
0.43 30
0.09 109
0.57 5
-0.17 39
0.21 14
-0.5 108
0.29 76
-0.67 7
-0.04 29
0.41 2
0.08 191
0.59 3
-0.33 53
-0.43 2
0.15 36
0.6 21
0.4 86
-0.45 2
0.14 199
0.61 4
-0.02 41
0.28 5
0.76 2
0.26 9
0.62 2
-0.22 6
-0.73 2
-0.01 36
0.56 8
0.22 41
0.49 2
-0.75 2
0.18 41
0.51 2
0.53 5
0.64 7
-0.86 2
-0.14 23
0.45 11
-0.3 2
-0.15 2
0.55 4
-0.1 15
0.9 5
0.46 3
0.33 756
-0.21 2
-0.08 29
0.66 2
0.82 5
0.23 18
-0.29 6
-0.38 2
0.01 102
0.02 120
0.63 6
0.78 5
0.17 289
-0.57 3
-0.18 3
0.74 2
0.3 13
0.94 3
-0.07 18
0.04 112
-0.44 2
0.83 3
0.35 7
-0.05 31
-0.2 18
0.91 2
0.03 144
-0.27 2
-0.4 8
0.07 127
0.31 15
0.36 16
-0.11 17
0.27 16
0.16 19
0.58 2
0.86 5
0.77 3
-0.13 31
-0.23 3
0.88 2
-0.12 4
0.54 3
0.32 5
0.37 3
-0.19 5
-0.03 45
-0.16 4
0.06 159
0.2 336
-1.0 166
-0.25

In [134]:
# 印出來看看那些平均不是0,1或-1的使用者是哪些
for u,i in user_dict.items():
    i = map(int, i.translate(None, string.punctuation))
    h= float(reduce(lambda q,p: p+q, i)/float(len(i)))
#     if len(str(h).split('.')[1])>0:
    if round(h,2)!=0.00 and h!=1.00 and h!=-1.00:
        print u,h

1861844987370232 0.5
670011473148809 0.5
599618733560958 0.0178571428571
1844563122469188 0.333333333333
1005797012809597 0.25
572666046228439 0.0714285714286
1474680899218232 0.666666666667
1307915375903383 0.2
738352869647545 0.0416666666667
1350911454981693 0.333333333333
10154255601663898 0.333333333333
10206497785417051 0.5
858274110970407 0.166666666667
410586812611180 0.2
606931662818484 0.166666666667
2202814306610736 0.25
1246204462139549 0.0434782608696
1404281279586067 0.142857142857
1330721280283389 0.5
1801174730113280 0.0526315789474
10202716045339364 0.25
654178704731563 0.4
1383072328401591 0.0833333333333
10209718422929659 0.25
571693833041040 0.222222222222
10154366147789337 0.0645161290323
309165636096968 0.0833333333333
1111376648921681 0.142857142857
1741013149560639 0.0625
1412602452094002 0.181818181818
391428771227128 0.5
397918770558608 0.357142857143
1737373013209228 0.0625
1278763078873307 0.125
515898278594157 0.16049382716
997406987043882 0.157894736842
124

In [122]:
# 找出某個使用者的留言
for i, c in enumerate(db.detector_yes.find({'user_id':'670011473148809'})):
    print i, c['comment']

0 小英總統加油~台灣加油
1 民進黨一定要贏!


##標一則，其他篇自動標記

In [26]:
## 修改中，可以運作，但是會timeout（因為網路慢？）
## 為什麼一篇一篇標反而比較快？
##去看mongoDB怎麼寫得比較efficient

import string
tsai_sup=['蔡總統加油','小英總統','英英總統','蔡英文總統加油','女總統加油','阿英總統加油','蔡總統我支持你','小英總統我支持你','英英總統我支持你','蔡英文總統我支持你','女總統我支持你','阿英總統我支持你']
tsai_not_sup=['台奸蔡英文','英英美代子','白賊英','蔡英文下台','蔡英文爛死了','菜英文','空心菜','祭央乂','暴力小英','武則英','尤達大師','女版馬英九','雙面蔡','蔡渣','18趴小英','亡國小英']


for c in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017}}):
    try:
#先找這個使用者有沒有在detector中出現過
        if db.detector.find({'user_id':c['user_id'].encode('utf-8')}).count()==0:
        #把標點符號拿掉，看有沒有出現tsai_sup中的詞
#                 有的話
            if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
                db.detector.insert({'comment':c['comment'],
                                        'user_id':c['user_id'],
                                        'user_name':c['user_name'],
                                        'comment_id':c['comment_id'],
                                        'post_id':c['post_id'],
                                        'page_name':facebook_page_name,
                                        'page_id':facebook_id,
                                        'comment_count':c['comment_count'],
                                        'like_count':c['like_count'],
                                        'stand':1,
                                        'auto':0})
#                 把這個user其他的留言也標記為這個stand
                for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']})[1:]:
                    db.detector.insert({'comment':uu['comment'],
                                        'user_id':c['user_id'],
                                        'user_name':c['user_name'],
                                        'comment_id':uu['comment_id'],
                                        'post_id':uu['post_id'],
                                        'page_name':facebook_page_name,
                                        'page_id':facebook_id,
                                        'comment_count':uu['comment_count'],
                                        'like_count':uu['like_count'],
                                        'stand':1,
                                        'auto':1})
                db.user.insert({'user_id':c['user_id'],
                               'user_name':c['user_name'],
                               'sup':facebook_page_name,
                               'n_sup':'',
                               'netural':''})

#                  不然
            elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in tsai_not_sup):
                n_sup.append(c['user_id'])
                db.detector.insert({'comment':c['comment'],
                                        'user_id':c['user_id'],
                                        'user_name':c['user_name'],
                                        'comment_id':c['comment_id'],
                                        'post_id':c['post_id'],
                                        'page_name':facebook_page_name,
                                        'page_id':facebook_id,
                                        'comment_count':c['comment_count'],
                                        'like_count':c['like_count'],
                                        'stand':-1,
                                        'auto':0})
#                 把這個user其他的留言也標記為這個stand
                for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']})[1:]:
                    db.detector.insert({'comment':uu['comment'],
                                        'user_id':c['user_id'],
                                        'user_name':c['user_name'],
                                        'comment_id':uu['comment_id'],
                                        'post_id':uu['post_id'],
                                        'page_name':facebook_page_name,
                                        'page_id':facebook_id,
                                        'comment_count':uu['comment_count'],
                                        'like_count':uu['like_count'],
                                        'stand':-1,
                                        'auto':1})
                db.user.insert({'user_id':c['user_id'],
                   'user_name':c['user_name'],
                   'sup':'',
                   'n_sup':facebook_page_name,
                   'netural':''})

#                 如果都沒有偵測到的話
            else:
                pass

        else: #在detector中有找到
            pass 
    except:
        print c['user_id'],len(c['comment'])


# 來處理到最後都被標記為0的user 

for c in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017}}):
    if db.detector.find({'user_id':c['user_id'].encode('utf-8')}).count()==0:
        for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']}):
            db.detector.insert({'comment':uu['comment'],
                                'user_id':c['user_id'],
                                'user_name':c['user_name'],
                                'comment_id':uu['comment_id'],
                                'post_id':uu['post_id'],
                                'page_name':facebook_page_name,
                                'page_id':facebook_id,
                                'comment_count':uu['comment_count'],
                                'like_count':uu['like_count'],
                                'stand':0,
                                'auto':1})
        db.user.insert({'user_id':c['user_id'],
                       'user_name':c['user_name'],
                       'sup':'',
                       'n_sup':'',
                       'netural':facebook_page_name})
        

db.detector.count()
db.update
# comments.update()
print db.detector.count()
client.close()
# db.posts.find({})

CursorNotFound: cursor id '169048340004' not valid at server

In [ ]:
import string
tsai_sup=['蔡總統加油','小英總統','英英總統','蔡英文總統加油','女總統加油','阿英總統加油','蔡總統我支持你','小英總統我支持你','英英總統我支持你','蔡英文總統我支持你','女總統我支持你','阿英總統我支持你']
tsai_not_sup=['英英美代子','白賊英','蔡英文下台','蔡英文爛死了','菜英文下台','菜英文爛死了','菜英文','空心菜','祭央乂','暴力小英','武則英','尤達大師','女版馬英九','雙面蔡','蔡渣','18趴小英','亡國小英']
# not_sure_1:['小英','阿英','小英總統','阿英總統','女總統','蔡總統','蔡英文','蔡英文總統','英英','英英總統']
# not_sure_2:['加油','讚','我愛你','支持你','我們支持你','全家支持你','我支持你']
    
sup = []  
n_sup = []
n = []
cannot = []


try:
    for c in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017}}):
        try:
    #先找這個使用者有沒有在detector中出現過
            if db.detector.find({'user_id':c['user_id'].encode('utf-8')}).count()==0:
            #把標點符號拿掉，看有沒有出現tsai_sup中的詞
#                 有的話
                if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
                    sup.append(c['user_id'])
                    db.detector.insert({'comment':c['comment'],
                                            'user_id':c['user_id'],
                                            'user_name':c['user_name'],
                                            'comment_id':c['comment_id'],
                                            'post_id':c['post_id'],
                                            'page_name':facebook_page_name,
                                            'page_id':facebook_id,
                                            'comment_count':c['comment_count'],
                                            'like_count':c['like_count'],
                                            'stand':1,
                                            'auto':0})
#                 把這個user其他的留言也標記為這個stand
                    for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']})[1:]:
                        db.detector.insert({'comment':uu['comment'],
                                            'user_id':c['user_id'],
                                            'user_name':c['user_name'],
                                            'comment_id':uu['comment_id'],
                                            'post_id':uu['post_id'],
                                            'page_name':facebook_page_name,
                                            'page_id':facebook_id,
                                            'comment_count':uu['comment_count'],
                                            'like_count':uu['like_count'],
                                            'stand':1,
                                            'auto':1})

#                  不然
                elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in not_list):
                    n_sup.append(c['user_id'])
                    db.detector.insert({'comment':c['comment'],
                                            'user_id':c['user_id'],
                                            'user_name':c['user_name'],
                                            'comment_id':c['comment_id'],
                                            'post_id':c['post_id'],
                                            'page_name':facebook_page_name,
                                            'page_id':facebook_id,
                                            'comment_count':c['comment_count'],
                                            'like_count':c['like_count'],
                                            'stand':-1,
                                            'auto':0})
#                 把這個user其他的留言也標記為這個stand
                    for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']})[1:]:
                        db.detector.insert({'comment':uu['comment'],
                                            'user_id':c['user_id'],
                                            'user_name':c['user_name'],
                                            'comment_id':uu['comment_id'],
                                            'post_id':uu['post_id'],
                                            'page_name':facebook_page_name,
                                            'page_id':facebook_id,
                                            'comment_count':uu['comment_count'],
                                            'like_count':uu['like_count'],
                                            'stand':-1,
                                            'auto':1})


#                 如果都沒有偵測到的話
                else:
                    n.append(c['user_id'])
#                     db.detector.insert({'comment':c['comment'],
#                                     'user_id':c['user_id'],
#                                     'user_name':c['user_name'],
#                                     'comment_id':c['comment_id'],
#                                     'post_id':c['post_id'],
#                                     'page_name':facebook_page_name,
#                                     'page_id':facebook_id,
#                                     'comment_count':c['comment_count'],
#                                     'like_count':c['like_count'],
#                                     'stand':0,
#                                     'auto':0})

            else:
                                #把之前有出現過的同為同一個留言者的都標記為跟已存在的那個一樣
                try:
    #                 如果有找到sup/not_sup的話  （上面標記過了）
                    if db.detector.find({'user_id':c['user_id'].encode('utf-8'),'stand':0}).count==0: #沒找到中立
                        pass
#                             db.detector.insert({'comment':c['comment'],
#                                                 'user_id':c['user_id'],
#                                                 'user_name':c['user_name'],
#                                                 'comment_id':c['comment_id'],
#                                                 'post_id':c['post_id'],
#                                                 'page_name':facebook_page_name,
#                                                 'page_id':facebook_id,
#                                                 'comment_count':c['comment_count'],
#                                                 'like_count':c['like_count'],
#                                                 'stand':-1,
#                                                 'auto':1})
#                             for dd in db.detector.find({'user_id':c['user_id'].encode('utf-8'),'stand':'not_sup'}):
#                                 try:
#                                     dd.update_many({'stand':'not_sure'}, {'stand':'changed_not_sup'})
#                                     sup_list.append(c['user_id'])
#                                 except:
#                                     pass

#                         else:
#                             if db.detector.find({'user_id':c['user_id'].encode('utf-8'),'stand':'sup'}).count!=0:
#                                 db.detector.insert({'comment':c['comment'],
#                                                         'user_id':c['user_id'],
#                                                         'user_name':c['user_name'],
#                                                         'comment_id':c['comment_id'],
#                                                         'post_id':c['post_id'],
#                                                         'page_name':facebook_page_name,
#                                                         'page_id':facebook_id,
#                                                         'comment_count':c['comment_count'],
#                                                         'like_count':c['like_count'],
#                                                         'stand':1,
#                                                         'auto':1})
#                             for dd in db.detector.find({'user_id':c['user_id'].encode('utf-8'),'stand':'not_sup'}):
#                                 try:
#                                     dd.update_many({'stand':'not_sure'}, {'stand':'changed_sup'})
#                                     nsup_list.append(c['user_id'])
#                                 except:
#                                     pass 
    #                 沒有的話就是中立的了
                    else:
                        print c['user_id']
#                         if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
#                             sup_list.append(c['user_id'])
#                             db.detector.insert({'comment':c['comment'],
#                                                     'user_id':c['user_id'],
#                                                     'user_name':c['user_name'],
#                                                     'comment_id':c['comment_id'],
#                                                     'post_id':c['post_id'],
#                                                     'page_name':facebook_page_name,
#                                                     'page_id':facebook_id,
#                                                     'comment_count':c['comment_count'],
#                                                     'like_count':c['like_count'],
#                                                     'stand':1,
#                                                     'auto':0})
#                             for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']}):
#                                 db.detector.insert({'comment':uu['comment'],
#                                                     'user_id':c['user_id'],
#                                                     'user_name':c['user_name'],
#                                                     'comment_id':uu['comment_id'],
#                                                     'post_id':uu['post_id'],
#                                                     'page_name':facebook_page_name,
#                                                     'page_id':facebook_id,
#                                                     'comment_count':uu['comment_count'],
#                                                     'like_count':uu['like_count'],
#                                                     'stand':1,
#                                                     'auto':1})

                        
#                         elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in not_list):
#                             nsup_list.append(c['user_id'])
#                             db.detector.insert({'comment':c['comment'],
#                                                     'user_id':c['user_id'],
#                                                     'user_name':c['user_name'],
#                                                     'comment_id':c['comment_id'],
#                                                     'post_id':c['post_id'],
#                                                     'page_name':facebook_page_name,
#                                                     'page_id':facebook_id,
#                                                     'comment_count':c['comment_count'],
#                                                     'like_count':c['like_count'],
#                                                     'stand':-1,
#                                                     'auto':0})
#                             for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':c['user_id']}):
#                                 db.detector.insert({'comment':uu['comment'],
#                                                     'user_id':c['user_id'],
#                                                     'user_name':c['user_name'],
#                                                     'comment_id':uu['comment_id'],
#                                                     'post_id':uu['post_id'],
#                                                     'page_name':facebook_page_name,
#                                                     'page_id':facebook_id,
#                                                     'comment_count':uu['comment_count'],
#                                                     'like_count':uu['like_count'],
#                                                     'stand':-1,
#                                                     'auto':1})


#                         else:
#                             n.append(c['user_id'])
#                             db.detector.insert({'comment':c['comment'],
#                                         'user_id':c['user_id'],
#                                         'user_name':c['user_name'],
#                                         'comment_id':c['comment_id'],
#                                         'post_id':c['post_id'],
#                                         'page_name':facebook_page_name,
#                                         'page_id':facebook_id,
#                                         'comment_count':c['comment_count'],
#                                         'like_count':c['like_count'],
#                                         'stand':0,
#                                         'auto':0})
                except:
                    print c['comment_id']
        except:
            print c['user_id']
except:
    cannot.append(c['user_id'])
        

db.detector.count()
db.update
# comments.update()
print db.detector.count()
print 'sup',len(sup)
print 'not_sup',len(n_sup)
print 'net',len(n)
print cannot
# db.posts.find({})

1099965543446028
1748533165426032
594716717373869
1068818039869791
728857077290186
594716717373869
728857077290186
1427175797306286
594716717373869
1249005045188639
2214887335402307
1724705961174365
320310488366344
166562877173813
239742623150841
118624741987994
170220723467016
118624741987994
156622918174248
118624741987994
239742623150841
170220723467016
239742623150841
420275468316005
1748533165426032
1566272933389040
1311199922230956
1241057279305058
1174054569282654
420275468316005
282233232193179
1305219206167227
1572805076080214
1421558964543539
1391244007612343
1174054569282654
1307524815962119
827893404008279
1307524815962119
594716717373869
1307524815962119
1421558964543539
1307524815962119
1103946969670940
170220723467016
170220723467016
282233232193179
282233232193179
1717549945223615
1380703991946957
481529768711382
373797766332119
373797766332119
171221206700169
217591135370819
217591135370819
1181118235284895
156383538198473
481529768711382
360211927711265
42924459408557

In [8]:
print len(sup)

2576


In [14]:
# 用append的方法--> 太慢了
import string
tsai_sup=['蔡總統加油','小英總統','英英總統','蔡英文總統加油','女總統加油','阿英總統加油','蔡總統我支持你','小英總統我支持你','英英總統我支持你','蔡英文總統我支持你','女總統我支持你','阿英總統我支持你']
tsai_not_sup=['英英美代子','白賊英','蔡英文下台','蔡英文爛死了','菜英文下台','菜英文爛死了','菜英文','空心菜','祭央乂','暴力小英','武則英','尤達大師','女版馬英九','雙面蔡','蔡渣','18趴小英','亡國小英']
    
sup = []  
n_sup = []
n = []
cannot = []

for c in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017}}):
    if c['user_id'] not in sup and c['user_id'] not in n_sup and c['user_id'] not in n:
    #把標點符號拿掉，看有沒有出現tsai_sup中的詞
        if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
            sup.append(c['user_id'])
        elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in not_sup):
            n_sup.append(c['user_id'])
        else:
            n.append(c['user_id'])

    elif c['user_id']in n: 
#                 如果有找到stand是中立的，就去判斷他是支持還是不支持或是中立
        if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
            sup.append(c['user_id'])
        elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in not_sup):
            n_sup.append(c['user_id'])
        else:
            n.append(c['user_id'])
    else:
        if c['user_id'] in sup or c['user_id'] in n_sup:
            pass

for su in sup:
    for d in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':su}):
        db.detector.insert({'comment':d['comment'],
                        'user_id':d['user_id'],
                        'user_name':d['user_name'],
                        'comment_id':d['comment_id'],
                        'post_id':d['post_id'],
                        'page_name':facebook_page_name,
                        'page_id':facebook_id,
                        'comment_count':d['comment_count'],
                        'like_count':d['like_count'],
                        'stand':1,})      

        db.detector.count()
        db.update

for ns in n_sup:
    for d in db.comments.find({'page_name':'Tsai','year':{'$gte':2015},'year':{'$lte':2017},'user_id':ns}):
        db.detector.insert({'comment':d['comment'],
                        'user_id':d['user_id'],
                        'user_name':d['user_name'],
                        'comment_id':d['comment_id'],
                        'post_id':d['post_id'],
                        'page_name':facebook_page_name,
                        'page_id':facebook_id,
                        'comment_count':d['comment_count'],
                        'like_count':d['like_count'],
                        'stand':-1,})      

        db.detector.count()
        db.update
                              
# comments.update()
print db.detector.count()
print 'sup',len(sup)
print 'not_sup',len(n_sup)
print 'net',n
# print cannot
# db.posts.find({})

KeyboardInterrupt: 

In [6]:
print len(n)

0


##用shell找mongoDB有重複的
####https://www.wiredprairie.us/blog/index.php/archives/1857

In [19]:
for c in db.comments.find({'comment_id':'10151280569516065_10850237'}):
    print c['comment']


馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
